In [10]:
## https://github.com/sundyCoder/IS_MS_SS
import os, sys
import imageio
import numpy as np
import argparse
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch
import cv2 as cv
import glob as glob
from numpy import clip

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tarfile

from six.moves import urllib
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import imageio

In [13]:
data_dir = '.Gen_dataset/VAE/MNIST/'

data = torch.load(os.path.join(data_dir,'generated_data.pt'))

In [25]:
data_list = [x.detach().numpy() for x in data['x']]
print(len(data_list))

10000


In [26]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

In [28]:
def preds2score(preds, splits=10):
    scores = []
    for i in range(splits):
        part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
        kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
        kl = np.mean(np.sum(kl, 1))
        scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)


def get_inception_score_mnist(images):
    num_splits=10
    batch_size = 100
    model_dir = './models/checkpoints/mnist_model_10.ckpt'
    splits = num_splits
    inps = []
    input_transform = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    for img in images:
        img = img.astype(np.float32)
        inps.append(np.expand_dims(img, 0))
    preds = []
    #pred_acc = []
    n_batches = int(math.ceil(float(len(inps)) / float(batch_size)))
    n_preds = 0

    net = ResNet18().cuda()
    net.load_state_dict(torch.load(model_dir))
    print("load model successfully")

    for i in range(n_batches):
        sys.stdout.write(".")
        sys.stdout.flush()
        inp = inps[(i * batch_size):min((i + 1) * batch_size, len(inps))]
        inp = np.concatenate(inp, 0)
        inp = np.expand_dims(inp, axis=1)
        inp = torch.from_numpy(inp).cuda()
        outputs = net(inp)
        pred = outputs.data.tolist()
        pred = softmax(pred)
        preds.append(pred)
        n_preds += outputs.shape[0]
    preds = np.concatenate(preds, 0)
    preds = np.exp(preds) / np.sum(np.exp(preds), 1, keepdims=True)
    mean_, std_ = preds2score(preds, splits)
    return mean_, std_

In [29]:
mean, std = get_inception_score_mnist(data_list)
print('\nInception mean: ', mean)
print('Inception std: ', std)


load model successfully
....................................................................................................
Inception mean:  1.0063228150435726
Inception std:  0.0037650017228050324


Inception score for CIFAR-10

In [11]:
data_dir = './Gen_datset/VAE/CIFAR-10/'

data = torch.load(os.path.join(data_dir,'generated_data.pt'))

In [12]:
cifar_data_list = [x.detach().numpy() for x in data['x']]
print(len(cifar_data_list))

10000


In [6]:
MODEL_DIR = '/tmp/imagenet'
DATA_URL = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
softmax = None

In [13]:
def get_inception_score_cifar(images):
  num_splits = 2
  tensor_layout = 'NHWC'
  splits = num_splits
  layout = tensor_layout

  assert(type(images) == list)
  assert(type(images[0]) == np.ndarray)
  assert(len(images[0].shape) == 3)
  print(images[0].min(), images[0].max(), images[0].dtype)
  # assert(np.max(images[0]) > 10)
  # assert(np.min(images[0]) >= 0.0)
  inps = []
  for img in images:
    img = img.astype(np.float32)
    inps.append(np.expand_dims(img, 0))
  bs = 100
  with tf.Session() as sess:
    preds = []
    n_batches = int(math.ceil(float(len(inps)) / float(bs)))
    n_preds = 0
    for i in range(n_batches):
        sys.stdout.write(".")
        sys.stdout.flush()
        inp = inps[(i * bs):min((i + 1) * bs, len(inps))]
        inp = np.concatenate(inp, 0)
        if layout == 'NCHW':
          inp = inp.transpose(0, 2, 3, 1)
        pred = sess.run(softmax, {'ExpandDims:0': inp})
        preds.append(pred)
        n_preds += pred.shape[0]
        print('Ran %d / %d images' % (n_preds, len(images)))    
    preds = np.concatenate(preds, 0)
    #preds = np.exp(preds) / np.sum(np.exp(preds), 1, keepdims=True)
    scores = []
    for i in range(splits):
      part = preds[(i * preds.shape[0] // splits):((i + 1) * preds.shape[0] // splits), :]
      kl = part * (np.log(part) - np.log(np.expand_dims(np.mean(part, 0), 0)))
      kl = np.mean(np.sum(kl, 1))
      scores.append(np.exp(kl))
    return np.mean(scores), np.std(scores)

# This function is called automatically.
def _init_inception():
  global softmax
  if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
  filename = DATA_URL.split('/')[-1]
  filepath = os.path.join(MODEL_DIR, filename)
  if not os.path.exists(filepath):
    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' % (
          filename, float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()
    filepath, _ = urllib.request.urlretrieve(DATA_URL, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    print('Succesfully downloaded', filename, statinfo.st_size, 'bytes.')
  tarfile.open(filepath, 'r:gz').extractall(MODEL_DIR)
  with tf.gfile.FastGFile(os.path.join(
      MODEL_DIR, 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')
  # Works with an arbitrary minibatch size.
  with tf.Session() as sess:
    pool3 = sess.graph.get_tensor_by_name('pool_3:0')
    ops = pool3.graph.get_operations()
    for op_idx, op in enumerate(ops):
        for o in op.outputs:
            shape = o.shape
            shape = list(shape)
            new_shape = []
            for j, s in enumerate(shape):
                if s == 1 and j == 0:
                    new_shape.append(None)
                else:
                    new_shape.append(s)
            o.__dict__['_shape_val'] = tf.TensorShape(new_shape)
            #o._shape = tf.TensorShape(new_shape)
    w = sess.graph.get_operation_by_name("softmax/logits/MatMul").inputs[1]
    logits = tf.matmul(tf.squeeze(pool3,[1,2]), w)
    #sslogits = tf.matmul(tf.squeeze(pool3), w)
    softmax = tf.nn.softmax(logits)


if softmax is None:
  _init_inception()

In [9]:
mean, std = get_inception_score_cifar(cifar_data_list)
print('Inception mean: ', mean)
print('Inception std: ', std)

0.41666007 0.76279896 float32
.

2023-06-12 12:05:37.810358: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled


Ran 100 / 10000 images
.Ran 200 / 10000 images
.Ran 300 / 10000 images
.Ran 400 / 10000 images
.Ran 500 / 10000 images
.Ran 600 / 10000 images
.Ran 700 / 10000 images
.Ran 800 / 10000 images
.Ran 900 / 10000 images
.Ran 1000 / 10000 images
.Ran 1100 / 10000 images
.Ran 1200 / 10000 images
.Ran 1300 / 10000 images
.Ran 1400 / 10000 images
.Ran 1500 / 10000 images
.Ran 1600 / 10000 images
.Ran 1700 / 10000 images
.Ran 1800 / 10000 images
.Ran 1900 / 10000 images
.Ran 2000 / 10000 images
.Ran 2100 / 10000 images
.Ran 2200 / 10000 images
.Ran 2300 / 10000 images
.Ran 2400 / 10000 images
.Ran 2500 / 10000 images
.Ran 2600 / 10000 images
.Ran 2700 / 10000 images
.Ran 2800 / 10000 images
.Ran 2900 / 10000 images
.Ran 3000 / 10000 images
.Ran 3100 / 10000 images
.Ran 3200 / 10000 images
.Ran 3300 / 10000 images
.Ran 3400 / 10000 images
.Ran 3500 / 10000 images
.Ran 3600 / 10000 images
.Ran 3700 / 10000 images
.Ran 3800 / 10000 images
.Ran 3900 / 10000 images
.Ran 4000 / 10000 images
.Ran 4100 